### Base line model

In [4]:
# !pip uninstall torch torchvision torchaudio huggingface transformers

In [5]:
#### CPU setup
# !pip install huggingface transformers torch torchvision

#### GPU setup
# !nvcc --version
# !pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

In [6]:
import torch 
from transformers import pipeline  # Use a pipeline as a high-level helper
import os # For all os level functions
import json # For parsing the json files


In [3]:
import torch
print(torch.version.cuda)


11.7


#### Util functions



In [33]:
### Setup the questions and answers for the image files
def obtain_questions_for_image_ids(image_ids: list, question_file_path: str= "questions.json"):
    """This function obtains the questions for the list of the image ids passed from the file"""
    with open(question_file_path) as file:
        questions_data = json.load(file)
    result = []
    for question_details in questions_data['questions']:
        if question_details['image_id'] in image_ids:
            result.append(question_details)
    return result


def obtain_annotations_for_image_ids(image_ids: list, annotations_file_path: str = "annotations.json"):
    """This function obtians the annotations for the list of the image ids passed from the file"""
    with open(annotations_file_path, 'r') as file:
        annotations_data = json.load(file)
    result = []
    for annotation_details in annotations_data['annotations']:
        if annotation_details['image_id'] in image_ids:
            result.append(annotation_details)
    return result


def get_refined_results_from_lists(relevant_questions:list, relevant_annotations:list):
    """This provides refined format of the question, image, plausible answers and other meta info"""
    result = []
    for question_info in relevant_questions:
        q_id = question_info['question_id']
        question = question_info['question']
        # print(f"Question ID: {q_id}, Question: {question}")
        for annotation_info in relevant_annotations:
            if annotation_info['question_id'] == q_id:
                answer = annotation_info["multiple_choice_answer"]
                plausile_answers = set(x['answer'] for x in annotation_info['answers'])
                result.append({"question_id": q_id, "question": question, 
                               "answer": answer, "plausible_answers": plausile_answers,
                               "image_id": annotation_info['image_id'], "question_type" :annotation_info['question_type'],
                               "answer_type": annotation_info['answer_type']})

    return result

def get_results(image_ids: list, folder: str):
    relevant_annotations = obtain_annotations_for_image_ids(image_ids, annotations_file_path=f"{folder}/annotations.json")
    relevant_questions = obtain_questions_for_image_ids(image_ids, question_file_path=f"{folder}/questions.json")
    return get_refined_results_from_lists(relevant_questions=relevant_questions, relevant_annotations=relevant_annotations)



In [34]:
validation_folder = "../data/validation"
validation_images_folder = f"{validation_folder}/images"

validation_image_files = [os.path.join(validation_images_folder, x) for x in os.listdir(validation_images_folder)]

sample_validation_image_files = validation_image_files[:10]

#### Setup for analysis

In [35]:
image_files = sample_validation_image_files
image_ids = [int(image_file.split(os.sep)[-1].split(".")[0].split("_")[-1]) for image_file in image_files]
image_ids

relevant_annotations = obtain_annotations_for_image_ids(image_ids, annotations_file_path=f"{validation_folder}/annotations.json")
relevant_questions = obtain_questions_for_image_ids(image_ids, question_file_path=f"{validation_folder}/questions.json")

results = get_results(image_ids, validation_folder)
print(image_ids)

[42, 73, 74, 133, 136, 139, 143, 164, 192, 196]


### Setup the baseline performance


In [37]:
## Obtain the captions for these image files
from transformers import pipeline
blipPipe = pipeline("image-to-text", model="Salesforce/blip-image-captioning-large")
generated_captions = blipPipe(image_files)


#     generated_captions = [{"image_caption": caption, "image_id": image_id, "image_file": image_file.split(os.sep)[-1]} 


/home/samavedam.m/.local/lib/python3.7/site-packages/transformers/generation/utils.py:1357: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [38]:
### Get the generated text along with image_id, image_file
captions = [x[0]['generated_text'] for x in generated_captions]

lookup = dict()
for caption, image_id, image_file in zip(captions, image_ids, image_files):
    lookup[image_id]= (image_file, caption)

In [21]:
### Hugging Face token for running the application

import os
from huggingface_hub import login

login(token=token) ## This is bound to fail, add your token from chat and run

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/samavedam.m/.cache/huggingface/token
Login successful


In [22]:
# Use a pipeline as a high-level helper
from transformers import pipeline

llamaPipe = pipeline("text-generation", model="meta-llama/Llama-2-7b-chat-hf")

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [19]:
### Zero Shot Performance
import tqdm


In [ ]:
answers = []
for instance in results:
    img_id = instance['image_id']
    question = instance['question']
    file, caption = lookup[img_id]
    prompt = f"Based on the image caption (generated by BLIP model): {caption}, Answer the question '{question}' \n Answer: "
    gen_text = llamaPipe(prompt)
    answers.append(gen_text)
# llamaPipe("What are the benefits of being vegetarian? Write a brief summary around the advantages and disadvantages of being vegetarian")

In [14]:
# for x in relevant_annotations:
#     print(f"multiple choice answer: {x['multiple_choice_answer']}")


In [ ]:
prompt_answers1 = []
for instance in results:
    img_id = instance['image_id']
    question = instance['question']
    file, caption = lookup[img_id]
    prompt = f"Based on the image caption (generated by BLIP model): {caption}. \n In plain simple English language without any emoticons or icons or font colors or punctuation marks, Answer in a single word the question: '{question}'. Remember if you are unable to answer the question based on the caption provided by BLIP, mention 'NA'.\nAnswer: "
    gen_text = llamaPipe(prompt)
    prompt_answers1.append(gen_text)
# llamaPipe("What are the benefits of being vegetarian? Write a brief summary around the advantages and disadvantages of being vegetarian")

In [42]:
results[0]['answer']

'white'

In [ ]:
answers

In [ ]:
#### ToDo 
### Write a function to evaluate the model performance as accuracy or the model

### Experiment with prompt to provide a single word answer.

### 